In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
#import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics_coref_new

In [3]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [35]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "macro_f1_score":   "$MACRO_F1",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision", "macro_f1_score" ] + params
        return df[cols]
    return df

In [36]:
def get_window_classifier_results(collections):
    dfs = []
    for col in collections:
        print(col)
        df = dict(get_df_sorted_by_f1score(col).iloc[0,:])
        df["Collection_" + col] = col
        dfs.append(df)
    return pd.DataFrame(dfs).sort_values("micro_f1_score", ascending=False)

# Which Problem Transformation Method Was Best?

In [37]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

## Coral Bleaching

In [38]:
df = get_window_classifier_results(["CB_TAGGING_VD_HYPER_PARAM_TUNING"])
df = round_data(df, 4)
df#["Collection_CB_TAGGING_VD,micro_f1_score,micro_recall,micro_precision".split(",")]

CB_TAGGING_VD_HYPER_PARAM_TUNING


,Collection_CB_TAGGING_VD_HYPER_PARAM_TUNING,macro_f1_score,micro_f1_score,micro_precision,micro_recall
0,CB_TAGGING_VD_HYPER_PARAM_TUNING,0.0,0.3166,0.4607,0.2412


## Skin Cancer

In [39]:
df = get_window_classifier_results(["SC_TAGGING_VD_HYPER_PARAM_TUNING"])
df = round_data(df, 4)
df#["Collection_SC_TAGGING_VD,micro_f1_score,micro_recall,micro_precision".split(",")]

SC_TAGGING_VD_HYPER_PARAM_TUNING


,Collection_SC_TAGGING_VD_HYPER_PARAM_TUNING,macro_f1_score,micro_f1_score,micro_precision,micro_recall
0,SC_TAGGING_VD_HYPER_PARAM_TUNING,0.0,0.3196,0.6433,0.2126


# Hyper Parameter Tuning Results

In [40]:
# Rows to print - df.head
ROWS = 5

- To get the TD and VD metrics, change the collection names below (VD->TD)

## Window Based Classifier - Hyper Parameter Tuning

### Coral Bleaching

In [41]:
params = "dual,C,penalty,fit_intercept,multi_class,window_size".split(",")
collection = "CB_TAGGING_VD_HYPER_PARAM_TUNING"

df = get_df_sorted_by_f1score(collection, params)

round_data(df.head(ROWS),4)
#df

,micro_f1_score,micro_recall,micro_precision,macro_f1_score,dual,C,penalty,fit_intercept,multi_class,window_size
0,0.3166,0.2412,0.4607,0.0,False,10.0,l2,True,ovr,9
1,0.3133,0.2382,0.4576,0.0,True,10.0,l2,True,ovr,9
2,0.3112,0.2412,0.4385,0.0,False,10.0,l1,True,ovr,9
3,0.3055,0.2471,0.4000,0.0,False,100.0,l2,True,ovr,9
4,0.2991,0.2059,0.5469,0.0,True,1.0,l2,True,ovr,9


### Skin Cancer

In [42]:
params = "dual,C,penalty,fit_intercept,multi_class,window_size".split(",")
collection = "SC_TAGGING_VD_HYPER_PARAM_TUNING"

df = get_df_sorted_by_f1score(collection, params)
#df = df[df.dual==True]
#df = df[df.C==0.5]
#df = df[df.penalty =='l2']
round_data(df.head(ROWS),3)

,micro_f1_score,micro_recall,micro_precision,macro_f1_score,dual,C,penalty,fit_intercept,multi_class,window_size
0,0.320,0.213,0.643,0.0,True,1.0,l2,True,ovr,5
1,0.298,0.221,0.459,0.0,True,10.0,l2,True,ovr,5
2,0.297,0.196,0.612,0.0,False,1.0,l1,True,ovr,5
3,0.292,0.223,0.424,0.0,False,10.0,l1,True,ovr,5
4,0.292,0.234,0.388,0.0,True,100.0,l2,True,ovr,5


# Test Set Performance

## <span style="color:red; font-weight:bold">TODO >>> </span>

## Window Based Classifier

### Coral Bleaching

In [43]:
# params = "dual,C,penalty,fit_intercept,multi_class,window_size".split(",")
# collection = "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS"

# df = get_df_sorted_by_f1score(collection, params)
# round_data(df.head(ROWS),3)

### Skin Cancer

In [44]:
# params = "dual,C,penalty,fit_intercept,multi_class,window_size".split(",")
# collection = "TEST_SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS"

# df = get_df_sorted_by_f1score(collection, params)
# round_data(df.head(ROWS),3)